In [7]:
import base64
import json
import pandas as pd
import requests
import tensorflow as tf

In [3]:
# Load dataset
df = pd.read_csv("data/data.csv")  # Ganti dengan path dataset Anda
test_text = df["Text"][0]  # Mengambil teks dari dataset untuk diuji

In [4]:

def prepare_json(text):
    """
    Fungsi ini mengubah teks menjadi format JSON yang sesuai untuk inferensi model.
    """
    feature_spec = {
        "Text": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})


In [5]:

def make_predictions(text):
    """
    Fungsi ini mengirim request ke model TensorFlow Serving dan mendapatkan prediksi.
    """
    json_data = prepare_json(text)

    # Ganti URL dengan endpoint model Anda
    endpoint = "http://localhost:8080/v1/models/racism-detection-model:predict"
    
    response = requests.post(endpoint, data=json_data)

    if response.status_code == 200:
        prediction = response.json()["predictions"][0][0]
        return prediction
    else:
        print("Error:", response.text)
        return None


In [6]:

# Melakukan prediksi
prediksi = make_predictions(test_text)

print(f"Teks: {test_text}")
print(f"Prediksi: {prediksi}")
print(f"Label Asli: {df['oh_label'][0]}")


Teks: @AAlwuhaib1977 Muslim mob violence against Hindus in Bangladesh continues in 2014. #Islam http://t.co/C1JBWJwuRc
Prediksi: 0.999851704
Label Asli: 1
